In [4]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [2]:
df_results = pd.read_parquet('results/seven-day-ahead_results.parquet')
df_results.head()

,date,actual_value,prediction,prediction_std,best_kernel,retrain
0,2024-12-24,"[[0.04927466634268923, -0.005146421043679282, ...","[[0.04332296863023677, -1.6948319141896206e-05...","[[0.4185319876412105, 0.4185319876412105, 0.41...",RationalQuadratic,True


In [3]:
df_one = pd.read_parquet('results/one-day-ahead/samples/2016-08-26.parquet')
df_one.head()

,beta0_future_val,beta1_future_val,beta2_future_val,lambda_future_val
0,0.050954,0.032897,-0.032895,0.228149
1,0.023140,0.012103,-0.009651,0.274028
2,0.057567,-0.025786,-0.038031,0.293598
3,0.095909,-0.056735,-0.016810,0.394400
4,0.018940,0.002185,-0.086265,0.336093


In [5]:
# Create features for the yield curves
def nelson_siegel(maturity, beta0, beta1, beta2, lambda_):
        return (beta0 +
                beta1 * ((1 - np.exp(-lambda_ * maturity)) / (lambda_ * maturity)) +
                beta2 * (((1 - np.exp(-lambda_ * maturity)) / (lambda_ * maturity)) - np.exp(-lambda_ * maturity)))


def nelson_siegel_params(yields, maturities):
    """
    Fit the Nelson-Siegel model to the yield curve data and return the parameters.
    Yields: array-like, bond yields
    Maturities: array-like, bond maturities in years
    """
    # Initial guess for the parameters
    initial_guess = [0.03, -0.02, 0.02, 0.5]
    # Fit the model
    params, _ = curve_fit(nelson_siegel, maturities, yields, p0=initial_guess)
    return params

In [9]:
maturieties = [round(1/12, 3), 1/4, 1/2, 1, 2, 3, 5, 7, 10, 20, 30]
# yields = bond_yields_train.loc['2015-06-15'].values / 100
# ns_params = nelson_siegel_params(yields, maturieties)
predicted_params = df_results['prediction'].values[0]
actual_params = df_results['actual_value'].values[0]
# make the predictions
predicted_yields = nelson_siegel(np.array(maturieties), *predicted_params)
actual_yields = nelson_siegel(np.array(maturieties), *actual_params)


TypeError: nelson_siegel() missing 3 required positional arguments: 'beta1', 'beta2', and 'lambda_'

In [8]:
predicted_params

array([array([ 4.33229686e-02, -1.69483191e-05, -1.89284566e-02,  6.83097607e-01])],
      dtype=object)